# ニューラルネットワークの学習

## 損失関数
- ２乗和誤差(回帰問題)
$$
    E = \frac{1}{2} \sum_{k}(y_k - t_k)^2
$$
- 交差エントロピー誤差(分類問題)
$$
    E = -\sum_{k}(t_k\log(y_k))
$$


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

###２乗和誤差###
def sum_squared_erro(y, t):
    return 0.5*np.sum((y-t)**2)

###交差エントロピー誤差###
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t=t.reshape(1, t.size)
        y=y.reshape(1, y.size)
        
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))/y.shape[0]



### ミニバッチ学習

In [ ]:
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.utils import to_categorical

"""
###データセットのダウンロード(本書のライブラリを写経するのが面倒だったのでkerasで代用)###
(x_train, t_train), (x_test, t_test) = mnist.load_data()
t_train = np.eye(N=10)[t_train.astype('int32').flatten()]   #どちらも同じ処理．one hot labelにしてる
t_test = to_categorical(t_test)
x_train.reshape(x_train.shape[0], 28*28)
x_test.reshape(x_test.shape[0], 28*28)

###ミニバッチを作成###
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]
"""


### 数値微分

In [ ]:
###微分###
def numerical_diff(f, x):
    h = 1e-4
    return (f(x+h)-f(x-h))/(2*h)

###勾配(各変数の偏微分を表したベクトル)###
def numerical_gradient_1d(f, x):
    h=1e-4
    grad = np.zeros_like(x.size)

    for idx in range(x.size):
        tmp = x[idx]

        x[idx] = tmp+h
        fx1 = f(x)

        x[idx] = tmp-h
        fx2 = f(x)
    
        grad[idx] = (fx1-fx2)/(2*h)
        x[idx] = tmp

    return grad

def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 値を元に戻す
        it.iternext()   
        
    return grad


###勾配降下法###
def gradient_descent(f, init_x, lr=0.01, step_num=100):

    x=init_x
    for i in range(step_num):
        x -= lr * numerical_gradient_1d(f, x)

    return x

In [ ]:
from common.functions import softmax, sigmoid

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3)  #ガウス分布で初期化

    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        y = softmax(self.predict(x))
        return cross_entropy_error(y, t)
    

In [ ]:
net = simpleNet()

x=np.array([0.6, 0.9])
p=net.predict(x)
t=np.array([0, 0, 1])
print(net.loss(x, t))

def f(W):
    return net.loss(x, t)

dW = numerical_gradient(f, net.W)
print(dW)



### 2層ニューラルネットワークの実装
- ミニバッチを作成
- 重みパラメータの勾配の算出
- パラメータの更新
- 繰り返す

In [ ]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params ={}
        self.params["W1"] = weight_init_std * np.random.rand(input_size, hidden_size)
        self.params["b1"] = np.zeros(hidden_size)
        self.params["W2"] = weight_init_std * np.random.rand(hidden_size, output_size)
        self.params["b2"] = np.zeros(output_size)

    def predict(self, x):
        z1 = sigmoid(np.dot(x, self.params["W1"]) + self.params["b1"])
        z2 = softmax(np.dot(z1, self.params["W2"]) + self.params["b2"])
        return z2
    
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy       

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t) 

        grads = {}
        grads["W1"] = numerical_gradient(loss_W, self.params["W1"])
        grads["W2"] = numerical_gradient(loss_W, self.params["W2"])
        grads["b1"] = numerical_gradient(loss_W, self.params["b1"])
        grads["b2"] = numerical_gradient(loss_W, self.params["b2"])

        return grads

In [ ]:
train_loss_list = []

(x_train, t_train), (x_test, t_test) = mnist.load_data()
t_train = np.eye(N=10)[t_train.astype('int32').flatten()]   #どちらも同じ処理．one hot labelにしてる
t_test = to_categorical(t_test)
x_train = x_train.astype("float32")/255.


iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

x_train = x_train.reshape([x_train.shape[0], 784])
x_test = x_test.reshape([x_test.shape[0], 784])
print(x_train.shape)
iter_per_epoch = max(train_size/batch_size, 1)

train_acc_list = []
test_acc_list = []

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    grad = network.numerical_gradient(x_batch, t_batch)

    for key in ("W1", "b1", "W2", "b2"):
        network.params[key] -= learning_rate*grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i%iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)